Problem Statement: Creating a Movie recommendation system which provides list of similar movies by analysing ratings provided by the user

Importing necessary libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mpl

Loading the dataset which contains list of movies and users who have watched those movies

In [2]:
movies_title_df=pd.read_csv('Movie_Id_Titles')

FileNotFoundError: [Errno 2] No such file or directory: 'Movie_Id_Titles'

In [ ]:
movies_title_df.head()

In [ ]:
movies_title_df.tail()

In [ ]:
movies_title_df.shape

Getting numerical statistics for the movies dataset

In [ ]:
movies_title_df.describe()

Checking if any null values present in the dataset

In [ ]:
movies_title_df.info()

Loading the ratings dataset which contains ratings provided by the users for the movies listed

In [ ]:
ratings_df=pd.read_csv('u.data',sep='\t',names=['user_id','item_id','ratings','timestamp'])
ratings_df.head()

Dropping the timestamp column which is not necessary for further analysis

In [ ]:
ratings_df.drop(['timestamp'],axis=1,inplace=True)
ratings_df.head()

Getting numerical statistics for the ratings dataset

In [ ]:
ratings_df.describe()

Checking if any null values present in ratings dataset

In [ ]:
ratings_df.info()

Merging two datasets with item_id as common column

In [ ]:
ratings_df=pd.merge(movies_title_df,ratings_df,on='item_id')

In [ ]:
ratings_df.head()

Getting numerical statistics with respect to ratings

In [ ]:
ratings_df.groupby('title').describe()

Getting the mean and count column alone and used for further analysis

In [ ]:
ratings_df_mean=ratings_df.groupby('title')['ratings'].describe()['mean']

In [ ]:
ratings_df_count=ratings_df.groupby('title')['ratings'].describe()['count']

In [ ]:
ratings_df_mean_count=pd.concat([ratings_df_mean,ratings_df_count],axis=1)
ratings_df_mean_count.reset_index()

Visualizing the mean in a histogram

In [ ]:
plt.rcParams['figure.figsize']=[10,5]
ratings_df_mean_count['mean'].plot(kind='hist',bins=100)

Nearly 10 movies were rated 5 stars were rated only once.This seems like only one person has given 5 star rating to those movies
Many people around 115 has given 3 stars to most of the movies

Nearly 10 movies were rated 5 stars that were listed below

In [ ]:
ratings_df_mean_count[ratings_df_mean_count['mean']==5]


Visualizing the count using histogram

In [ ]:
plt.rcParams['figure.figsize']=[10,5]
ratings_df_mean_count['count'].plot(kind='hist',bins=100,color='r')

This shows that only few movies around 50 were rated many number of ratings 

Highly rated movies

In [ ]:
ratings_df_mean_count.sort_values('count',ascending=False).head(150)

Low rated movies

In [ ]:
ratings_df_mean_count.sort_values('count',ascending=True).head(150)

Creating matrix of ratings given by every user with respect to each movie

In [ ]:
user_id_ratings_matrix=ratings_df.pivot_table(index='user_id',columns='title',values='ratings')
user_id_ratings_matrix

User Ratings for the movie StarWars and Titanic is shown below

In [ ]:
starwars=user_id_ratings_matrix['Star Wars (1977)']
starwars

In [ ]:
titanic=user_id_ratings_matrix['Titanic (1997)']
titanic

Correlation is exhibited between titanic and all other movies

In [ ]:
titanic_correlation=pd.DataFrame(user_id_ratings_matrix.corrwith(titanic),columns=['Correlation'])
titanic_correlation

In [ ]:
titanic_correlation=titanic_correlation.join(ratings_df_mean_count['count'])
titanic_correlation

Removing null values in correlation column

In [ ]:
titanic_correlation.dropna(inplace=True)
titanic_correlation

Sorting based on correlation values

In [ ]:
titanic_correlation.sort_values('Correlation',ascending=False)

Top 5 movies which are highly correlated with titanic

In [ ]:
titanic_correlation[titanic_correlation['count']>80].sort_values('Correlation',ascending=False).head()

Correlation is exhibited between Star Wars and other movies

In [ ]:
starwars_correlation=pd.DataFrame(user_id_ratings_matrix.corrwith(starwars),columns=['Correlation'])
starwars_correlation

Removing null values in correlation column

In [ ]:
starwars_correlation.dropna(inplace=True)

In [ ]:
starwars_correlation=starwars_correlation.join(ratings_df_mean_count['count'])
starwars_correlation

Top 5 movies which are highly correlated with starwars

In [ ]:
starwars_correlation=starwars_correlation[starwars_correlation['count']>80].sort_values('Correlation',ascending=False)
starwars_correlation.head()

Creating correlation matrix between all the movies using pearson's coefficient

In [ ]:
movies_correlation=user_id_ratings_matrix.corr(method='pearson',min_periods=80)
movies_correlation

Loading the user ratings list

In [ ]:
myratings=pd.read_csv('My_Ratings.csv')
myratings

Checking movies provided by user with respect to the movie list for similar ones

In [ ]:
similarmovies_list=pd.Series()
for i in range(len(myratings)):
    similarmovies=movies_correlation[myratings['Movie Name'][i]].dropna()
    similarmovies=similarmovies.map(lambda x:x*myratings['Ratings'][i])
    similarmovies_list=similarmovies_list.append(similarmovies)
    
   

List of movies similar to 'Liar Liar (1997)' is listed below

In [ ]:
similarmovies_list.sort_values(inplace=True,ascending=False)
similarmovies_list.head(10)

List of movies similar to 'Star Wars' is given below

In [ ]:
similarmovies_list=pd.Series()
for i in range(1,2):
    similarmovies=movies_correlation[myratings['Movie Name'][i]].dropna()
    similarmovies=similarmovies.map(lambda x:x*myratings['Ratings'][i])
    similarmovies_list=similarmovies_list.append(similarmovies)

In [ ]:
similarmovies_list.sort_values(inplace=True,ascending=False)
similarmovies_list.head(10)

In [ ]:
similarmovies_list=pd.Series()
for i in range(2,3):
    similarmovies=movies_correlation[myratings['Movie Name'][i]].dropna()
    similarmovies=similarmovies.map(lambda x:x*myratings['Ratings'][i])
    similarmovies_list=similarmovies_list.append(similarmovies)

List of similar movies to Titanic(1997)

In [ ]:
similarmovies_list.sort_values(inplace=True,ascending=False)
similarmovies_list.head(10)

Conclusion: Based on the user rating for several movies given in the Movies_Id_Titles dataset we can get list of similar movies based on Pearson's Coeffient